In [40]:
import numpy as np
from pymatgen.core import Structure, PeriodicSite, DummySpecie
# from pymatgen.analysis.graphs import StructureGraph
from pymatgen.analysis.local_env import MinimumDistanceNN

In [ ]:
'''# Load defective structure
defective_struct = Structure.from_file("low_density_defects/MoS2/MoS2cifs/6141cf1a1d648932fbc34297.cif")
struct_lattice = defective_struct.lattice

# Get reference structure
ref_unit_cell = Structure.from_file("low_density_defects/MoS2/MoS2.cif")
reference_struct = ref_unit_cell.make_supercell([8,8,1])'''


/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/core/structure.py:3087: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 48 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [41]:
# Load defective structure
defective_struct = Structure.from_file("/home/adamkim/PROGRAMMING/Dataset/high_density_defects/MoS2_500/highMoS2cifs/MoS2_Mo56W4Se3S117_b593cd47-47e7-4a55-9a27-466f5c3f661c.cif")
struct_lattice = defective_struct.lattice

# Get reference structure
ref_unit_cell = Structure.from_file("/home/adamkim/PROGRAMMING/Dataset/high_density_defects/MoS2_500/MoS2.cif")
reference_struct = ref_unit_cell.make_supercell([8,8,1])


/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/core/structure.py:3087: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 45 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [42]:
def struct_to_dict(structure):
    list_of_sites = structure.sites
    list_of_frac_coords = np.round(structure.frac_coords,3)
    structure_dict = {i: j for i, j in zip(list_of_sites, list_of_frac_coords)}
    return structure_dict

In [43]:
def get_defects(defective_struct, reference_struct):
    copy_defective_struct = defective_struct.copy()
    # struct to dict
    defective_dict = struct_to_dict(copy_defective_struct)
    reference_dict = struct_to_dict(reference_struct)

    # Get lattice of defective structure
    structure_lattice = copy_defective_struct.lattice

    # Define list to add all defect sites
    defects_list = []

    # Define a dictionary to hold atomic numbers
    site_property = {}

    for ref_site, ref_coords in reference_dict.items():
        matching = False
        for def_site, def_coords in defective_dict.items():
            if np.array_equal(ref_coords, def_coords):
                matching = True
                if ref_site.specie != def_site.specie: # Substitution case
                    # Add site to defects list
                    defects_list.append(def_site)

                    # Get atomic number change and defect type
                    add_property = {"orig_new_an":(ref_site.specie.Z, def_site.specie.Z),
                                    "atomic_number_change": def_site.specie.Z - ref_site.specie.Z,
                                    "vacancy_defect": 0.0,
                                    "substitution_defect": 1.0}
                    site_property[def_site] = add_property
                else:
                    add_property = {"orig_new_an":(ref_site.specie.Z, def_site.specie.Z),
                                    "atomic_number_change": def_site.specie.Z - ref_site.specie.Z,
                                    "vacancy_defect": 0.0,
                                    "substitution_defect": 0.0}
                    site_property[def_site] = add_property

                    

        if not matching: # Vacancy case
            # Add site to defective structure
            vacant_site = PeriodicSite(
                species= DummySpecie(),
                coords= ref_coords,
                coords_are_cartesian= False, 
                lattice= structure_lattice
                )
            
            copy_defective_struct.append(vacant_site.species, vacant_site.frac_coords)

            # Add site to defects list
            defects_list.append(vacant_site)

            # Get atomic number change and defect type
            add_property={"orig_new_an": (ref_site.specie.Z, 0),
                          "atomic_number_change": 0 - ref_site.specie.Z,
                          "vacancy_defect": 1.0,
                          "substitution_defect": 0.0}
            site_property[vacant_site] = add_property

            

    # Till this point every defect site is in the defective structure
    return copy_defective_struct, defects_list, site_property


# Get defect structure
new_defective_struct, all_defects, defect_an_property = get_defects(defective_struct, reference_struct)


In [44]:
print(new_defective_struct)

Full Formula (X12 Mo56 W4 Se3 S117)
Reduced Formula: X12Mo56W4(SeS39)3
abc   :  25.522526  25.522526  20.000000
angles:  90.000000  90.000000 120.000000
pbc   :       True       True       True
Sites (192)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Mo    0.041667  0.083333  0.185988
  1  Mo    0.041667  0.208333  0.185988
  2  Mo    0.041667  0.333333  0.185988
  3  Mo    0.041667  0.458333  0.185988
  4  Mo    0.041667  0.583333  0.185988
  5  Mo    0.041667  0.708333  0.185988
  6  Mo    0.041667  0.833333  0.185988
  7  Mo    0.166667  0.083333  0.185988
  8  Mo    0.166667  0.333333  0.185988
  9  Mo    0.166667  0.583333  0.185988
 10  Mo    0.166667  0.708333  0.185988
 11  Mo    0.166667  0.833333  0.185988
 12  Mo    0.166667  0.958333  0.185988
 13  Mo    0.291667  0.083333  0.185988
 14  Mo    0.291667  0.208333  0.185988
 15  Mo    0.291667  0.333333  0.185988
 16  Mo    0.291667  0.458333  0.185988
 17  Mo    0.291667  0.583333  0.1

In [45]:
print(all_defects)

[PeriodicSite: X0+ (-11.15, 21.17, 3.72) [0.042, 0.958, 0.186], PeriodicSite: W56 (W) (1.595, 4.605, 3.72) [0.1667, 0.2083, 0.186], PeriodicSite: X0+ (-1.582, 10.12, 3.72) [0.167, 0.458, 0.186], PeriodicSite: W57 (W) (1.595, 15.66, 3.72) [0.4167, 0.7083, 0.186], PeriodicSite: X0+ (19.15, 1.835, 3.72) [0.792, 0.083, 0.186], PeriodicSite: W58 (W) (17.55, 4.605, 3.72) [0.7917, 0.2083, 0.186], PeriodicSite: X0+ (15.96, 7.36, 3.72) [0.792, 0.333, 0.186], PeriodicSite: W59 (W) (22.33, 1.842, 3.72) [0.9167, 0.08333, 0.186], PeriodicSite: X0+ (-0.01276, 3.691, 2.16) [0.083, 0.167, 0.108], PeriodicSite: Se60 (Se) (-4.785, 11.97, 2.155) [0.08333, 0.5417, 0.1077], PeriodicSite: X0+ (3.178, 3.691, 2.16) [0.208, 0.167, 0.108], PeriodicSite: Se61 (Se) (3.19, 14.74, 2.155) [0.4583, 0.6667, 0.1077], PeriodicSite: Se62 (Se) (3.19, 20.26, 2.155) [0.5833, 0.9167, 0.1077], PeriodicSite: X0+ (17.53, 0.9283, 2.16) [0.708, 0.042, 0.108], PeriodicSite: X0+ (19.13, 9.217, 2.16) [0.958, 0.417, 0.108], PeriodicS

In [46]:
def get_properties(new_defective_struct, all_defect_sites):
    list_of_sites = new_defective_struct.sites

    # Define an instance responsible for getting the nearest neighbors
    cnn = MinimumDistanceNN(cutoff=3.0)

    # dictionary of properties
    site_dict = {} # Of each site
    new_dict = {}  # Of all sites

    # Lets get the properties for each defect site
    for i in list_of_sites:
        the_index = list_of_sites.index(i)

        # Get the nearest neighbors of each defect site
        neighbors = cnn.get_nn_info(new_defective_struct, the_index)

        # Get the coordination numbers
        coord_numbers = len(neighbors)

        # Get the vital attributes:
        # neighbors
        the_neighbors = []
        # site index of neighbor
        neighbors_site_index = []
        # neighbors image
        neighbors_image = []
        # distance to neighbors
        bl_to_neighbors = []
        # weights to neighbors
        weights_to_neighbors = []
        # atomic numbers of neighbors
        elements_an = []

        for neighbor_info in neighbors:
            the_neighbors.append(neighbor_info['site'])
            neighbors_site_index.append(neighbor_info['site_index'])
            neighbors_image.append(neighbor_info['image'])
            bond_length = neighbor_info['site'].distance(i)
            bl_to_neighbors.append(bond_length)
            weights_to_neighbors.append(neighbor_info['weight'])
            # Handle the atomic numbers
            the_an = neighbor_info["site"].specie.Z
            if the_an > 0 and the_an < 119:
                elements_an.append(the_an)
            else:
                the_an = 0
                elements_an.append(the_an)
                
        # Get distance to other defect sites
        dist_to_other_defects = []
        for n in all_defect_sites:
            dist = i.distance(n)
            dist_to_other_defects.append(dist)

        # Collect these properties as dictionaries
        site_dict[i] = {"nearest_neighbors": the_neighbors,
                        "neighbors_atomic_numbers": elements_an,
                        "nearest_neighbors_index": neighbors_site_index,
                        "neighbors_image": neighbors_image,
                        "neighbors_weight": weights_to_neighbors,
                        "distance_to_neighbors": bl_to_neighbors,
                        "coordination_number": coord_numbers,
                        "distance_to_defects": dist_to_other_defects}
        
        new_dict.update(site_dict)
    
    return site_dict


In [ ]:
sites_properties = get_properties(new_defective_struct, all_defects)

for key1 in sites_properties.keys():
    if key1 in defect_an_property.keys():
        sites_properties[key1].update(defect_an_property[key1])
    else:
        pass

for site_struct in new_defective_struct.sites:
    if site_struct in sites_properties.keys():
        site_struct.properties.update(sites_properties[site_struct])
    else:
        pass

print(new_defective_struct)